In [1]:
from datasets import load_dataset
dataset = load_dataset("Someman/hindi-summarization")

d:\Software\Anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
print(len(text))

30488242


In [43]:
text = ""
numRows = 0
for row in dataset['train']:
    datatext = row['article']
    numRows = numRows + 1
    text = text + datatext
    # print(datatext)
    # if numRows % 1 == 0:
    #     print(numRows)
    if numRows > 5000:
        break

print(len(text))
# print(text[0:2917])

13910763


In [44]:
# Removing unnecessary characters and also replacing multi spaces with single space
import re
characters_to_remove = "\r\n"
min_length = 5
def cleanData(text):
    for character in characters_to_remove:
        text = text.replace(character, " ")
    text = re.sub(r'\s+', ' ', text)
    # print(text)
    return text

def removeDuplicateSentences(sentences):
    updatedsentences = list(set(sentences))
    updatedsentences = [s for s in updatedsentences if len(s) >= min_length]
    updatedsentences = [re.sub(r'^\s+', '', s) for s in updatedsentences if len(s) >= min_length]
    return updatedsentences

In [45]:
newtext = cleanData(text)
sep = "।"
sentences = [x+sep for x in newtext.split(u"।")]
print(len(sentences))
updatedsentences = removeDuplicateSentences(sentences)
print(len(updatedsentences))
# print(sentences)
# print(updatedsentences)

72863
27143


In [46]:
print(updatedsentences[0])

काफिले में शमिल कोंटा विधायक कवासी लखमा के पीएसओ को भी गोली लगने की जानकारी सामने आ रही है।


In [47]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?[\u0900-\u0963\u0965-\u09FF]+| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

# print(re.findall(gpt2pat, "इस वाक्य का अनुवाद गूगल100 ट्रांसलेट द्वारा किया जा रहा है।😅"))

In [49]:
words = []
for sentence in updatedsentences:
    words.extend(re.findall(gpt2pat, sentence))

In [51]:
print(words[0:50])
print(len(words))

['काफिले', ' में', ' शमिल', ' कोंटा', ' विधायक', ' कवासी', ' लखमा', ' के', ' पीएसओ', ' को', ' भी', ' गोली', ' लगने', ' की', ' जानकारी', ' सामने', ' आ', ' रही', ' है', '।', 'टिप्पणियां', ' पुलिस', ' ने', ' व्यक्ति', ' को', ' गिरफ्तार', ' कर', ' मामला', ' दर्ज', ' कर', ' लिया', ' है', '।', 'टिप्पणियां', ' उत्तर', ' गुजरात', ' की', ' पाटण', ' विधानसभा', ' से', ' जहां', ' 2002', ' और', ' 2007', ' में', ' आनंदीबेन', ' पटेल', ' ने', ' चुनाव', ' जीता']
1998486


In [52]:
wordTokens = []
for i, word in enumerate(words):
    # print(word)
    tokens = word.encode("utf-8") # raw bytes
    tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
    wordTokens.append(tokens)
    # print(len(wordTokens))
    # print(len(wordTokens[i]))
    # if i > 10:
    #     break
    # print(wordTokens)
# print(wordTokens)

In [53]:
print(len(wordTokens))

1998486


In [55]:
def getLen(wordTokens):
    totalLen = 0
    totalLen = sum([len(word) for word in wordTokens])
    return totalLen

getLen(wordTokens)

22419678

In [57]:
def get_stats_vocab(words):
    counts = {}
    for ids in words:
      for pair in zip(ids, ids[1:]):
          counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge_vocab(words, pair, idx):
  newWords = []
  for ids in words:
    i = 0
    newids = []
    while i < len(ids):
      if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
        newids.append(idx)
        i += 2
      else:
        newids.append(ids[i])
        i += 1
    newWords.append(newids)
  
  return newWords

# ---
vocab_size = 5000 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(wordTokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats_vocab(ids)
  # sorted_stats = sorted(((v,k) for k,v in stats.items()), reverse=True)
  # # print("sorted stats:", sorted_stats)
  # if (sorted_stats[0][0] < 2):
  #   break
  # print("Stats:", stats)
  pair = max(stats, key=stats.get)
  
  idx = 256 + i
  if i % 50 == 0:
    print(f"merging {pair} into a new token {idx}")
  ids = merge_vocab(ids, pair, idx)
  merges[pair] = idx

print("tokens length:", getLen(wordTokens))
print("ids length:", getLen(ids))
print(f"compression ratio: {getLen(wordTokens) / getLen(ids):.2f}X")

merging (224, 164) into a new token 256
merging (262, 270) into a new token 306
merging (256, 136) into a new token 356
merging (32, 49) into a new token 406
merging (275, 282) into a new token 456
merging (269, 149) into a new token 506
merging (270, 308) into a new token 556
merging (105, 116) into a new token 606
merging (485, 260) into a new token 656
merging (275, 260) into a new token 706
merging (264, 261) into a new token 756
merging (459, 287) into a new token 806
merging (280, 395) into a new token 856
merging (267, 518) into a new token 906
merging (321, 316) into a new token 956
merging (317, 395) into a new token 1006
merging (270, 504) into a new token 1056
merging (264, 308) into a new token 1106
merging (532, 277) into a new token 1156
merging (490, 855) into a new token 1206
merging (451, 417) into a new token 1256
merging (282, 308) into a new token 1306
merging (331, 277) into a new token 1356
merging (282, 312) into a new token 1406
merging (540, 436) into a new tok

In [59]:
print(wordTokens[0:30])
print(ids[0:10])

[[224, 164, 149, 224, 164, 190, 224, 164, 171, 224, 164, 191, 224, 164, 178, 224, 165, 135], [32, 224, 164, 174, 224, 165, 135, 224, 164, 130], [32, 224, 164, 182, 224, 164, 174, 224, 164, 191, 224, 164, 178], [32, 224, 164, 149, 224, 165, 139, 224, 164, 130, 224, 164, 159, 224, 164, 190], [32, 224, 164, 181, 224, 164, 191, 224, 164, 167, 224, 164, 190, 224, 164, 175, 224, 164, 149], [32, 224, 164, 149, 224, 164, 181, 224, 164, 190, 224, 164, 184, 224, 165, 128], [32, 224, 164, 178, 224, 164, 150, 224, 164, 174, 224, 164, 190], [32, 224, 164, 149, 224, 165, 135], [32, 224, 164, 170, 224, 165, 128, 224, 164, 143, 224, 164, 184, 224, 164, 147], [32, 224, 164, 149, 224, 165, 139], [32, 224, 164, 173, 224, 165, 128], [32, 224, 164, 151, 224, 165, 139, 224, 164, 178, 224, 165, 128], [32, 224, 164, 178, 224, 164, 151, 224, 164, 168, 224, 165, 135], [32, 224, 164, 149, 224, 165, 128], [32, 224, 164, 156, 224, 164, 190, 224, 164, 168, 224, 164, 149, 224, 164, 190, 224, 164, 176, 224, 165, 128]

In [60]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

print(len(vocab))

5000


In [61]:
print(merges)

{(224, 164): 256, (32, 256): 257, (224, 165): 258, (256, 190): 259, (258, 135): 260, (256, 176): 261, (257, 149): 262, (259, 256): 263, (258, 128): 264, (256, 191): 265, (258, 141): 266, (256, 130): 267, (265, 256): 268, (266, 256): 269, (258, 139): 270, (256, 168): 271, (257, 184): 272, (256, 149): 273, (257, 185): 274, (257, 174): 275, (256, 185): 276, (256, 164): 277, (256, 184): 278, (257, 170): 279, (258, 136): 280, (256, 178): 281, (258, 129): 282, (260, 267): 283, (257, 168): 284, (262, 260): 285, (256, 174): 286, (259, 261): 287, (257, 172): 288, (266, 261): 289, (274, 280): 290, (257, 156): 291, (257, 166): 292, (275, 283): 293, (256, 170): 294, (256, 151): 295, (256, 166): 296, (257, 133): 297, (270, 267): 298, (257, 176): 299, (257, 178): 300, (257, 181): 301, (175, 259): 302, (263, 168): 303, (262, 264): 304, (256, 181): 305, (262, 270): 306, (258, 130): 307, (256, 159): 308, (271, 260): 309, (284, 260): 310, (256, 172): 311, (256, 161): 312, (257, 151): 313, (261, 269): 31

In [62]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  i = 0
  newids = []
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  
  return newids

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([256, 170, 256, 176, 258, 133, 256, 171]))

def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode("पर अफगानिस्तान में इस्तेमाल"))

परॅफ
[1430, 4011, 293, 1734]


In [63]:
text = "हमने उन्हें विशेष क्षेत्रों के बारे में बता दिया है। हमने उनसे कदम उठाने (इस दिशा में) की अपील की है।"

In [64]:
print(len(list(text.encode("utf-8")))/len(encode(text)))

10.36


In [65]:
print(len(list(text.encode("utf-8"))))

259


In [66]:
encodedText = encode("हमने उन्हें विशेष क्षेत्रों के बारे में बता दिया है। हमने उनसे कदम उठाने (इस दिशा में) की अपील की है।")
print(encodedText)

[3658, 688, 1613, 3221, 285, 901, 293, 1184, 479, 290, 328, 1911, 2039, 1538, 4063, 397, 646, 2868, 293, 41, 304, 2617, 304, 290, 328]


In [67]:
decode(encodedText)

'हमने उन्हें विशेष क्षेत्रों के बारे में बता दिया है। हमने उनसे कदम उठाने (इस दिशा में) की अपील की है।'

In [68]:
strTokens = []
for text in encodedText:
    strTokens.append(decode([text]))

In [69]:
print(strTokens)

['हमने', ' उन्हें', ' विशेष', ' क्षेत्रों', ' के', ' बारे', ' में', ' बता', ' दिया', ' है', '।', ' हमने', ' उनसे', ' कदम', ' उठाने', ' (', 'इस', ' दिशा', ' में', ')', ' की', ' अपील', ' की', ' है', '।']


In [71]:
import pickle

with open('mergesOld.pkl', 'wb') as merges_file:
     pickle.dump(merges, merges_file)
with open('vocabOld.pkl', 'wb') as vocab_file:
     pickle.dump(vocab, vocab_file)

In [70]:
import json
with open('merges.json', 'w') as merges_file:
     merges_file.write(json.dumps(merges))
with open('vocab.json', 'w') as vocab_file:
     vocab_file.write(json.dumps(vocab))

TypeError: keys must be str, int, float, bool or None, not tuple

In [ ]:
with open('merges.json') as merges_json:
    newmerges = json.load(merges_json)
print(newmerges)

In [ ]:
print(vocab)

In [ ]:
with open('vocab.json') as vocab_json:
    newvocab = json.load(vocab_json)
print(newvocab)